# Sprint 8 Machine Learning

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

Desarrollar un modelo con la mayor exactitud posible. En este proyecto, el umbral de exactitud es 0.75. Usa el dataset para comprobar la exactitud.

## Abre y examina el archivo de datos.

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

display(df)
print()
print(df.info())

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


El DataFrame tiene 5 columnas: calls,minutes, messages, mb_used	y is_ultra. En este caso se eligío un modelo de clasificación porque el objetivo es predecir si un usuario debe usar el plan "Smart" (0) o "Ultra" (1), con esto se estaría trabajando con un problema de clasificación binaria. Además la columna target sería "is_ultra"

## Segmenta los datos fuente en un conjunto de entrenamiento, uno de validación y uno de prueba.

In [3]:
# Variable objetivo is_ultra — plan para el mes actual (Ultra - 1, Smart - 0).
# Separación: 60% entrenamiento, 20% validación y 20% prueba


features = df[['calls', 'minutes', 'messages', 'mb_used']]
target = df['is_ultra']

# División del conjunto de datos en dos: conjunto de prueba (20%) y el resto (conjunto de validación + conjuntos de entrenamiento) (80%)

features_80, features_test, target_80, target_test = train_test_split(features, target, test_size=0.20, random_state=12345)

# División del conjunto de datos restante en conjunto de entrenamiento (60%) y conjunto de validación (20%)

features_train, features_valid, target_train, target_valid = train_test_split(features_80, target_80, test_size=0.25, random_state=12345)


## Investiga la calidad de diferentes modelos cambiando los hiperparámetros. Describe brevemente los hallazgos del estudio.

Para la regresión Logistica se va a utilizar como parametro:
    
solver: Algoritmo a utilizar en el problema de optimización. Los valores posibles son:

- 'liblinear': Un solucionador de regresión logística bueno para conjuntos de datos pequeños.
- 'lbfgs': Un solucionador de optimización que soporta las regularizaciones L2.

In [4]:
# Definiendo rangos de los hiperparámetros
depth_opc = [1,2,3,4,5,6]
estimators_opc = [10,50,100]
solver_opc = ['liblinear', 'lbfgs']

best_accuracy = 0
best_model = None

# Logistic Regression

for solver in solver_opc:
    model_lg = LogisticRegression(random_state=54321, solver=solver)
    model_lg.fit(features_train, target_train)
    predictions_lg_valid = model_lg.predict(features_valid)
    result_lg_valid = accuracy_score(target_valid, predictions_lg_valid)
    if result_lg_valid > best_accuracy:
        best_accuracy = result_lg_valid
        best_model = model_lg
        best_params = {'model': 'LogisticRegression', 'solver': solver}

# Imprimir los mejores hiperparámetros
print(f'Mejores hiperparámetros: {best_params}')
print(f'Mayor Accuracy en el conjunto de validación: {best_accuracy}')





Mejores hiperparámetros: {'model': 'LogisticRegression', 'solver': 'liblinear'}
Mayor Accuracy en el conjunto de validación: 0.7293934681181959


In [5]:
# Decision Tree. Modificando el hiperparametro max_depth

for depth in depth_opc:
    model_dt = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_dt.fit(features_train, target_train)
    predictions_dt_valid = model_dt.predict(features_valid)
    result_dt_valid = accuracy_score(target_valid, predictions_dt_valid)
    if result_dt_valid > best_accuracy:
        best_accuracy = result_dt_valid
        best_model = model_dt
        best_params = {'model': 'DecisionTree', 'max_depth': depth}
 

print(f'Mejores hiperparámetros: {best_params}')
print(f'Mayor Accuracy en el conjunto de validación: {best_accuracy}')   


Mejores hiperparámetros: {'model': 'DecisionTree', 'max_depth': 3}
Mayor Accuracy en el conjunto de validación: 0.7651632970451011


In [6]:
# Random Forest

# Predicción y evaluación en el conjunto de validación
   
for est in estimators_opc:
    model_rf = RandomForestClassifier(random_state=54321, n_estimators=est)
    model_rf.fit(features_train, target_train)
    predictions_rf_valid = model_rf.predict(features_valid)
    result_rf_valid = accuracy_score(target_valid, predictions_rf_valid)
    if result_rf_valid > best_accuracy:
        best_accuracy = result_rf_valid
        best_model = model_rf
        best_params = {'model': 'RandomForest', 'n_estimators': est}

print(f'Mejores hiperparámetros: {best_params}')
print(f'Mayor Accuracy en el conjunto de validación: {best_accuracy}')

Mejores hiperparámetros: {'model': 'RandomForest', 'n_estimators': 100}
Mayor Accuracy en el conjunto de validación: 0.7978227060653188


El mejor modelo basado en la exactitud en el conjunto de validación es el Random Forest Classifier. Este modelo tiene la mayor exactitud (0.79) con hiperparámetros de n_estimators: 100, por lo tanto, es el modelo que se usará para ser evaluado en el conjunto de prueba.

Exactitud de los  modelos:

- Decision tree: 0.76  max_depth: 3
- Random Forest: 0.79  n_estimators: 100
- Logistic Regression: 0.72  solver: 'liblinear'

## Comprueba la calidad del modelo usando el conjunto de prueba.

In [7]:
# Predicción y evaluación en el conjunto de prueba


predict_test = best_model.predict(features_test)
result_test = accuracy_score(target_test, predict_test)
print(f'Nombre del mejor modelo en el conjunto de prueba: {best_model}')
print(f'Accuracy del mejor modelo en el conjunto de prueba: {result_test}')

print(best_model)
print(best_params['n_estimators'])

Nombre del mejor modelo en el conjunto de prueba: RandomForestClassifier(random_state=54321)
Accuracy del mejor modelo en el conjunto de prueba: 0.7916018662519441
RandomForestClassifier(random_state=54321)
100


Se comprueba que el modelo Random Forest es el modelo que tiene la mayor exactitud al estar por encima del umbral de 0.75 cuando se hace la prueba con un modelo de prueba, obteniendo una exactitud de 0.7916018662519441

## prueba de cordura al modelo.

In [8]:
if best_params['model'] == 'RandomForest':
    model_cordura = RandomForestClassifier(random_state=54321, n_estimators=best_params['n_estimators'])
elif best_params['model'] == 'DecisionTree':
    model_cordura = DecisionTreeClassifier(random_state=12345, max_depth=best_params['max_depth'])
else:
    model_cordura = LogisticRegression(random_state=54321, solver=best_params['solver'])
    
model_cordura.fit(features_train, target_train)
predict_valid_cordura = model_cordura.predict(features_valid)
result_model_cordura = accuracy_score(target_valid,predict_valid_cordura)

if result_model_cordura == best_accuracy:
    print(f'La prueba de cordura pasó: La precisión obtenida coincide con la mejor precisión registrada. Accuracy: {result_model_cordura}')
else:
    print("La prueba de cordura falló")

La prueba de cordura pasó: La precisión obtenida coincide con la mejor precisión registrada. Accuracy: 0.7978227060653188


El modelo de Random Forest Classifier pasó la prueba de cordura, comprobando que el modelo seleccionado tiene un buen rendimiento y es capaz de generalizar bien a nuevos datos. Esto significa que el modelo no está sobreajustando los datos de entrenamiento y es capaz de capturar patrones relevantes en los datos de prueba.

Esto indica que el modelo es efectivo para recomendar el plan correcto entre Smart y Ultra, con una exactitud por encima del umbral establecido de 0.75. La prueba de cordura ha sido superada, indicando la validez del modelo entrenado.